In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.utils import np_utils, plot_model
import cv2
from PIL import Image
import pandas as pd

Using TensorFlow backend.


In [4]:
#test_images_path = '/content/drive/My Drive/Colab Notebooks/AOI/test_images'
#test_images = os.listdir(test_images_path)
train_images_path = 'train_images'
train_images = os.listdir(train_images_path) 
#print("test_images size :",len(test_images))
print("train_images size :",len(train_images))

train_images size : 2528


In [5]:
#確定train_data裡的IMG是否為一樣大小
for i in train_images[:2] :
    img = cv2.imread(train_images_path+"/"+i)
    print(img.shape)
    if img.shape != (512,512,3) :
        print('ee')


(512, 512, 3)
(512, 512, 3)


In [6]:
train_images.sort()
trainDataPaths = []
for i in train_images:
    trainDataPaths.append(train_images_path+"/"+i)

In [7]:
pixel_depth = 255.0

def prep_data(images):
    count = len(images)
    data = np.ndarray((count, 512, 512, 3), dtype=np.float32)

    for i, image_file in enumerate(images) :
        img = cv2.imread(image_file)
        image_data = np.array(img, dtype=np.float32)
        #正規化
        image_data[:, :, 0] = (image_data[:, :, 0].astype(float) - pixel_depth / 2) / pixel_depth
        image_data[:, :, 1] = (image_data[:, :, 1].astype(float) - pixel_depth / 2) / pixel_depth
        image_data[:, :, 2] = (image_data[:, :, 2].astype(float) - pixel_depth / 2) / pixel_depth

        data[i] = image_data
        if i % 50 == 0:
            print("Processed {} of {}".format(i, count))

    return data

In [8]:
data = prep_data(trainDataPaths)

Processed 0 of 2528
Processed 50 of 2528
Processed 100 of 2528
Processed 150 of 2528
Processed 200 of 2528
Processed 250 of 2528
Processed 300 of 2528
Processed 350 of 2528
Processed 400 of 2528
Processed 450 of 2528
Processed 500 of 2528
Processed 550 of 2528
Processed 600 of 2528
Processed 650 of 2528
Processed 700 of 2528
Processed 750 of 2528
Processed 800 of 2528
Processed 850 of 2528
Processed 900 of 2528
Processed 950 of 2528
Processed 1000 of 2528
Processed 1050 of 2528
Processed 1100 of 2528
Processed 1150 of 2528
Processed 1200 of 2528
Processed 1250 of 2528
Processed 1300 of 2528
Processed 1350 of 2528
Processed 1400 of 2528
Processed 1450 of 2528
Processed 1500 of 2528
Processed 1550 of 2528
Processed 1600 of 2528
Processed 1650 of 2528
Processed 1700 of 2528
Processed 1750 of 2528
Processed 1800 of 2528
Processed 1850 of 2528
Processed 1900 of 2528
Processed 1950 of 2528
Processed 2000 of 2528
Processed 2050 of 2528
Processed 2100 of 2528
Processed 2150 of 2528
Processed 2

In [9]:
from sklearn.preprocessing import OneHotEncoder
dfLabel = pd.read_csv('train.csv')
np_label = np.array(dfLabel['Label'])
np_label = np_utils.to_categorical(np_label)
(x_train, x_test, y_train, y_test) = train_test_split(data, np_label, test_size=0.2, random_state=42)


In [10]:
from keras.models import Sequential, model_from_yaml, load_model
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.optimizers import Adam, SGD

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), padding='same', input_shape=(512, 512, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=36, kernel_size=(5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 512, 512, 16)      1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 256, 256, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 36)      14436     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 128, 36)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 128, 36)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 589824)           

In [11]:
sgd = Adam(lr=0.0001)
model.compile(loss='binary_crossentropy',optimizer=sgd, metrics=['accuracy'])


In [12]:
train_history = model.fit(x_train, y_train, batch_size=20, epochs=30, verbose=2, validation_data=(x_test, y_test))

Train on 2022 samples, validate on 506 samples
Epoch 1/30
 - 415s - loss: 0.3169 - accuracy: 0.8668 - val_loss: 0.2999 - val_accuracy: 0.8673
Epoch 2/30
 - 569s - loss: 0.2075 - accuracy: 0.9113 - val_loss: 0.2996 - val_accuracy: 0.8594
Epoch 3/30
 - 532s - loss: 0.1636 - accuracy: 0.9319 - val_loss: 0.2999 - val_accuracy: 0.8712
Epoch 4/30
 - 518s - loss: 0.1204 - accuracy: 0.9542 - val_loss: 0.4096 - val_accuracy: 0.8511
Epoch 5/30
 - 537s - loss: 0.1048 - accuracy: 0.9608 - val_loss: 0.2746 - val_accuracy: 0.8900
Epoch 6/30
 - 545s - loss: 0.0807 - accuracy: 0.9710 - val_loss: 0.3655 - val_accuracy: 0.8762
Epoch 7/30
 - 559s - loss: 0.0710 - accuracy: 0.9736 - val_loss: 0.3023 - val_accuracy: 0.8956
Epoch 8/30
 - 557s - loss: 0.0594 - accuracy: 0.9789 - val_loss: 0.2871 - val_accuracy: 0.9015
Epoch 9/30
 - 596s - loss: 0.0686 - accuracy: 0.9772 - val_loss: 0.2661 - val_accuracy: 0.9042
Epoch 10/30
 - 559s - loss: 0.0446 - accuracy: 0.9864 - val_loss: 0.3310 - val_accuracy: 0.8959
Ep

In [13]:
model.save('my_model.h5')

In [14]:
model.save_weights('my_model_weights.h5')